In [ ]:
import math
import random
import numpy as np
from scipy.stats import poisson
from scipy.stats import norm
n = 7
ch = 1
cp = 10
co = 320
cr = 20
max_inv_level = 250
min_inv_level =180
means=[-1]*n
__opt_cost = []
expected_cost = 0
for i in range(n):
    means[i] = round(50 * (1 + math.sin(math.pi * i /6))) + min(i, n-i)
init_inv = 0
threshold = 0.0001

max_demand = int(poisson.ppf(loc = 0 , mu = max(means), q = (1-threshold)))
prob = [[0.0 for _ in range(0, max_demand +1)] for _ in range(n)]
for i in range(n):
    for j in range(max_demand +1):
        if means[i] != 0 :
            prob[i][j] = poisson.pmf(k = j, mu = means[i])
        else:
            prob[i][j] = poisson.pmf(k = j, mu = 0.1)  # 这些都和上面的过程很像
    prob[i] = prob[i]/sum(prob[i])
max_inv_level = n * max_demand
min_inv_level = - n * max_demand + init_inv

demand = [0 for _ in range(n * max_demand + 1)]  # n个时期，最多可以累积n * max_demand的需求
for i in range(max_demand + 1):
    demand[i] = prob[0][i]
for i in range(1, n):
    temp = [0 for _ in range(n * max_demand + 1)]
    for a in range((n - 1) * max_demand + 1):
        for b in range(max_demand + 1):
            temp[a + b] += prob[i][b] * demand[a]  # 求出需求为a+b的概率
    demand = temp
j = n * max_demand
pdf_dem = demand[j]
while pdf_dem < threshold:  # 从后往前 将发生概率比较小的需求视为不可能事件
    j -= 1
    pdf_dem += demand[j - 1]
    
conv_prob = np.zeros((n, n, n * max_demand + 1))  # 创造三维数组

for i in range(n):
    for j in range(max_demand+1):
        conv_prob[i][i][j] = prob[i][j]
    # self.conv_prob[i][i] = self.conv_prob[i][i] / sum(self.conv_prob[i][i])

for i in range(0, n):
    for i2 in range(i+1, n):
        for j in range((i2-i)*max_demand +1):
            for j2 in range(max_demand+1):
                conv_prob[i][i2][j+j2] += conv_prob[i][i2-1][j] * prob[i2][j2]
        conv_prob[i][i2] = conv_prob[i][i2] / sum(conv_prob[i][i2])
        
__opt_cost = [[float("inf") for _ in range(min_inv_level, max_inv_level + 1)] for _ in range(n + 1)]
opt_q = [[max_inv_level for _ in range(min_inv_level, max_inv_level +1)] for _ in range(n+1)]
opt_r = [[0 for _ in range(min_inv_level, max_inv_level +1)] for _ in range(n+1)]     

for i in range(min_inv_level, max_inv_level +1):
    __opt_cost[n][i] = 0
    
    
def cost(t, i, r):
    temp = 0.0
    for d, p in enumerate(conv_prob[t][t + r - 1]):
        close_inv = i - d
        if close_inv <= min_inv_level:
            temp = temp + p * __opt_cost[t + r][min_inv_level]
        else:
            temp = temp + p * __opt_cost[t + r][close_inv]
    for j in range(r):
        for d, p in enumerate(conv_prob[t][t + j]):
            close_inv = i - d
            if close_inv >= 0:
                temp = temp + p * ch * close_inv
            if close_inv < 0:
                temp = temp + p * (-cp) * close_inv
    return temp    

for t in range(n - 1, -1, -1):
    for i in range(min_inv_level, max_inv_level + 1):
        bsf_cost = float("inf")
        bsf_q = float("inf")
        bsf_r = float("inf")

        q_limit = max_inv_level - i
        for r in range(1, n - t+1):
            for q in range(0, q_limit + 1):
                temp = cr if q == 0 else cr+co
                temp += cost(t,i+q,r)
                if temp < bsf_cost:
                    bsf_cost = temp
                    bsf_q = q
                    bsf_r = r
        __opt_cost[t][i] = bsf_cost
        opt_q[t][i] = bsf_q
        opt_r[t][i] = bsf_r
j = 0
if opt_q[0][init_inv] == 0:
    self.__opt_cost[0][init_inv] -= cr
    j = 1

expected_cost = __opt_cost[0][0]


s = [float("inf")] * n
S = [float("inf")] * n
R = [0] * n
for t in range(n):
    for i in range(0, max_inv_level +1):
        if opt_q[t][i] > 0:
            s[t] = i
            S[t] = i + opt_q[t][i]
while j < n:
    R[j] = 1
    j += opt_r[j][S[j]]

for i in range(n):
    if R[i] == 0:
        s[i] = float("inf")
        S[i] = float("inf")

In [ ]:
print("Reviews:\t" + str(R))
print("s:\t\t\t" + str(s))
print("S:\t\t\t" + str(S))

In [ ]:
print('1')